In [1]:
# -*- coding: utf-8 -*-
# ================================================================
# AMF Stress Test — workflow JOUR PAR JOUR (base = exposures)
# ================================================================
# Ce script :
# 1) charge le mapping (POS + scenario_paths),
# 2) charge 'exposures' (greeks) + nettoie/contrôle qualité (QC),
# 3) restreint les scénarios au périmètre d'exposures (par Identifier),
# 4) applique les chocs pour un jour donné (day_step_apply),
# 5) renvoie un 'exposures_next' prêt à être MODIFIÉ avant le jour suivant.
# ------------------------------------------------
# CLÉ DE MERGE / AGRÉGATION = Identifier  (ISIN conservé pour info)
# ------------------------------------------------

from pathlib import Path
import pandas as pd
import numpy as np
from collateral_management import process_pv_after_day_1, roll_balance_for_next_day

# =======================
# CONFIG — à adapter
# =======================
EXCEL_MAPPING_PATH = r"C:\Users\abenjelloun\OneDrive - Cooperactions\GAM-E-Risk Perf - RMP\1.PROD\1.REGLEMENTAIRE\14.Stress Test AMF (JB)\Production\Périmètre et positions\Matrices correspondance_AB.xlsx"
SHEET_POS = "Test_Aya"   # entêtes ligne 2 -> header=1
SHEET_SCEN =  "scenario_paths"
POS_HEADER_ROW = 1               # 0-based

EXPOSURES_PATH = r"C:\Users\abenjelloun\OneDrive - Cooperactions\GAM-E-Risk Perf - RMP\1.PROD\1.REGLEMENTAIRE\14.Stress Test AMF (JB)\Production\Périmètre et positions\GROUPAMA-BreakoutsOverTime-2025-03-31.csv"
TRIOPTIMA_PATH = r"C:\Users\abenjelloun\OneDrive - Cooperactions\GAM-E-Risk Perf - RMP\1.PROD\1.REGLEMENTAIRE\14.Stress Test AMF (JB)\Production\Périmètre et positions\search_groupama-am_2025-03-31.xlsx"
COLLATERAL_BALANCES_PATH = r"C:\Users\abenjelloun\OneDrive - Cooperactions\GAM-E-Risk Perf - RMP\1.PROD\1.REGLEMENTAIRE\14.Stress Test AMF (JB)\Production\Périmètre et positions\Collat_Cash_MTM_LU_20250401.csv" 
COLLATERAL_BALANCE_DAY_COL = "Balance J"
COLLATERAL_BALANCE_PREV_COL = "Balance J-1"
COLLATERAL_THRESHOLD_COL = "Seuil déclenchement"
FUTURE_ASSET_CLASSES = {
       'Bond Future',
        'Equity Index Future'
        'FX Future'}

# Jours possibles (doivent exister dans scenario_paths)
DAYS = ["Day 1","Day 2","Day 3","Day 4","Day 5","Day 10"]

# Règles de méthode
INCLUDE_OTHER_INFLATION_SWAP = True

# =======================
# Utils colonnes / texte
# =======================
def _clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Nettoie des colonnes Excel (espaces, 'Unnamed', points)."""
    new_cols = []
    for c in df.columns:
        if c is None or (isinstance(c, str) and c.lower().startswith("unnamed")):
            new_cols.append(None); continue
        s = str(c).strip().replace("\u00A0", " ")
        s = " ".join(s.split())
        s = s.replace(". ", " ").replace(".", " ")
        new_cols.append(s)
    df.columns = new_cols
    return df

def _norm_str(x):
    """Normalise légèrement une chaîne (pour Market / Variable)."""
    if pd.isna(x): return np.nan
    s = str(x).strip().replace("\u00A0", " ")
    s = " ".join(s.split()).replace(". ", " ").replace(".", " ")
    return s

# =======================
# Chargement mapping (POS + scen)
# =======================
POS_BASE_COLS = [
    "Identifier","ISIN","Counterparty","Description","Currency","AssetType","Sector1","Seniority",
    "CompositeBroadRating","MaturityDate","Maturity","Maturity Band","EffectiveMaturityDate",
    "LiquidityScore","Country","{Class_Rating}"
]
POS_MV_PAIRS = [(f"Market {i}", f"Variable {i}") for i in range(1, 7)]
SCEN_BASE_COLS = ["Market","Variable","Comment","Type","Unit","T0",
                  "Day 1","Day 2","Day 3","Day 4","Day 5","Day 10"]

def load_mapping(path_excel: str|Path) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Lit la feuille POS et scenario_paths."""
    xls = pd.ExcelFile(path_excel)
    pos  = pd.read_excel(xls, SHEET_POS, header=POS_HEADER_ROW)
    scen = pd.read_excel(xls, SHEET_SCEN)
    pos  = _clean_cols(pos)
    scen = _clean_cols(scen)
    return pos, scen

def melt_pos(pos: pd.DataFrame) -> pd.DataFrame:
    """Transforme POS en format long : 1 ligne par (Market k, Variable k) non vide."""
    pos = pos.copy()
    for mk, vk in POS_MV_PAIRS:
        if mk in pos.columns: pos[mk] = pos[mk].apply(_norm_str)
        if vk in pos.columns: pos[vk] = pos[vk].apply(_norm_str)

    base_cols = [c for c in POS_BASE_COLS if c in pos.columns]
    mv_pairs_present = [(mk,vk) for mk,vk in POS_MV_PAIRS if mk in pos.columns and vk in pos.columns]

    rows = []
    for _, row in pos.iterrows():
        base = {c: row.get(c, np.nan) for c in base_cols}
        for mk, vk in mv_pairs_present:
            market, variable = row[mk], row[vk]
            if pd.notna(market) and str(market) != "":
                rows.append({**base, "Market": market, "Variable": (variable if pd.notna(variable) else np.nan)})
    out = pd.DataFrame(rows)
    if "Identifier" not in out.columns: out["Identifier"] = np.arange(len(out))
    return out

def prepare_scenarios(scen: pd.DataFrame) -> pd.DataFrame:
    """Nettoie la table scenario_paths & conserve les colonnes utiles."""
    scen = scen.copy()
    scen = scen[[c for c in SCEN_BASE_COLS if c in scen.columns]]
    scen["Market"] = scen["Market"].apply(_norm_str)
    if "Variable" in scen.columns: scen["Variable"] = scen["Variable"].apply(_norm_str)
    if "Type" in scen.columns:     scen["Type"]     = scen["Type"].apply(lambda x: str(x).strip().lower() if pd.notna(x) else x)
    if "Unit" in scen.columns:     scen["Unit"]     = scen["Unit"].apply(lambda x: str(x).strip().lower() if pd.notna(x) else x)
    return scen

def merge_pos_scen(pos_long: pd.DataFrame, scen: pd.DataFrame) -> pd.DataFrame:
    """Relie POS (long) aux scénarios : jointure (Market,Variable) ; si Variable vide -> jointure sur Market seul."""
    left_mv = pos_long.dropna(subset=["Market","Variable"]) if "Variable" in pos_long.columns else pos_long.copy()
    mv_merge = left_mv.merge(scen, on=["Market","Variable"], how="left")
    if "Variable" in pos_long.columns: left_m = pos_long[pos_long["Variable"].isna()].copy()
    else:                               left_m = pd.DataFrame(columns=pos_long.columns)
    if not left_m.empty:
        m_merge = left_m.merge(scen.drop(columns=["Variable"], errors="ignore"), on="Market", how="left")
        return pd.concat([mv_merge, m_merge], ignore_index=True)
    return mv_merge

def available_days(scen: pd.DataFrame) -> list[str]:
    """Retourne la liste des colonnes 'Day n' disponibles."""
    return [c for c in scen.columns if isinstance(c, str) and c.lower().startswith("day")]

# =======================
# Limiter les scénarios au périmètre d'exposures (clé = Identifier)
# =======================
def restrict_scenarios_to_exposures(merged_mapping: pd.DataFrame,
                                    exposures: pd.DataFrame,
                                    key_col: str = "Identifier") -> pd.DataFrame:
    """Garde uniquement les lignes de mapping dont la clé existe dans exposures."""
    if key_col not in merged_mapping.columns or key_col not in exposures.columns:
        return merged_mapping
    keys = exposures[key_col].astype(str).unique()
    return merged_mapping[merged_mapping[key_col].astype(str).isin(keys)].copy()

# =======================
# Standardisation des chocs
# =======================
def standardize_shock(value, unit: str|None) -> float|None:
    """Choc standardisé en décimal: 50 bps -> 0.005 ; -10% -> -0.10 ; 2 p.p -> 0.02."""
    if pd.isna(value): return None
    try: val = float(value)
    except: return None
    if unit is None: return val
    u = unit.lower()
    if u in ["bp","bps"]: return val / 10_000.0
    if u in ["%","percent","percentage","p.p","pp","ppt","percentage point","percentage points"]:
        return val / 100.0
    return val

def _to_bps(shock_std: float, unit: str|None) -> float:
    """Convertit un choc standardisé en bps numériques si besoin (pour PV01 / CS01 / Infl01)."""
    if shock_std is None or pd.isna(shock_std) or unit is None: return np.nan
    u = unit.lower()
    if u in ["bp","bps"]: return shock_std * 10_000.0
    if u in ["%","percent","percentage","p.p","pp","ppt","percentage point","percentage points","pc"]:
        return shock_std * 10_000.0
    return np.nan

def build_daily_shocks(merged: pd.DataFrame, day_col: str) -> pd.DataFrame:
    """Prépare les chocs d'un jour (inclut Identifier & ISIN si présents)."""
    if day_col not in merged.columns:
        raise ValueError(f"Jour '{day_col}' introuvable. Jours dispo: {available_days(merged)}")
    out = merged.copy()
    out["shock_raw"] = out[day_col]
    out["shock_std"] = [standardize_shock(v, u) for v, u in zip(out["shock_raw"], out.get("Unit", pd.Series([None]*len(out))))]

    keep = ["Identifier","ISIN","Market","Variable","Type","Unit","T0", day_col, "shock_std","Comment"]
    keep = [c for c in keep if c in out.columns]
    return out[keep]

# =======================
# Chargement exposures + QC
# =======================
def load_exposures(path_excel: str|Path, return_qc: bool = True):
    """Charge le fichier d'expositions, nettoie et met des valeurs par défaut pour éviter les NaN."""
    df = pd.read_csv(path_excel,sep=';',decimal='.')
    df = _clean_cols(df)

    qc = {"path": str(path_excel)}
    qc["rows_raw"] = int(df.shape[0])

    # Convertir en numérique les colonnes clés si elles existent
    num_cols = [
        "TV","MacaulayDuration","Duration","DollarRateConvexity1pc",
        "RateDelta1bp","RateVega","SpreadDelta1bp","CreditVega",
        "EquityDelta","EquityGamma","EquityVega","FXDelta","FXVega","InflationDelta1bp",
        "Nominal","TVPercent"
    ]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # Enlever les lignes agrégées 'TOTAL' si jamais elles existent
    removed_total = 0
    if "AssetID" in df.columns:
        mask_total = df["AssetID"].astype(str).str.upper().eq("TOTAL")
        removed_total = int(mask_total.sum())
        df = df[~mask_total].copy()

    

    qc["rows_removed_total"] = removed_total
    qc["rows_after_filter"] = int(df.shape[0])

    mask_0 = df['Nominal'] == 0
    removed_n_0 = int(mask_0.sum())
    df = df[~mask_0].copy()

    qc['row_removes_notional_null'] = removed_n_0

    # Sensi NaN -> 0 (évite les effets NaN)
    sensi_zero = [
        "RateDelta1bp","SpreadDelta1bp","InflationDelta1bp",
        "EquityDelta","FXDelta","RateVega","CreditVega","EquityVega","FXVega","EquityGamma"
    ]
    qc["filled_zero"] = {}
    for c in sensi_zero:
        if c in df.columns:
            n = int(df[c].isna().sum())
            df[c] = df[c].fillna(0.0)
            qc["filled_zero"][c] = n

    # Duration NaN -> 0
    if "Duration" in df.columns:
        qc["duration_filled_zero"] = int(df["Duration"].isna().sum())
        df["Duration"] = df["Duration"].fillna(0.0)
    else:
        qc["duration_missing_col"] = True
        df["Duration"] = 0.0

    # DollarRateConvexity1pc : NaN -> 0
    if "DollarRateConvexity1pc" in df.columns:
        qc["dollarconvexity_filled_zero"] = int(df["DollarRateConvexity1pc"].isna().sum())
        df["DollarRateConvexity1pc"] = df["DollarRateConvexity1pc"].fillna(0.0)
    else:
        qc["dollarconvexity_missing_col"] = True
        df["DollarRateConvexity1pc"] = 0.0

    return (df, qc) if return_qc else df

def print_qc_report(qc: dict):
    """Affiche un petit rapport de nettoyage des expositions."""
    print(f"📄 Fichier : {qc.get('path','')}")
    print(f"📦 Lignes brutes : {qc['rows_raw']}")
    print(f"🧹 Lignes 'TOTAL' supprimées : {qc['rows_removed_total']}")
    print(f"✅ Lignes après filtre : {qc['rows_after_filter']}")
    print("🔧 NaN → 0 (sensis) :")
    for k, v in qc["filled_zero"].items():
        print(f"  - {k:<22}: {v}")
    if "duration_filled_zero" in qc:
        print(f"\n⏱  Duration NaN → 0 : {qc['duration_filled_zero']}")
    if qc.get("duration_missing_col"):
        print("⚠️  Colonne 'Duration' manquante → créée à 0")
    if qc.get("dollarconvexity_missing_col"):
        print("⚠️  Colonne 'DollarRateConvexity1pc' manquante → créée")
    if "dollarconvexity_filled_zero" in qc:
        print(f"📐 DollarRateConvexity1pc NaN → 0 : {qc['dollarconvexity_filled_zero']}")
        
def load_counterparty_mapping(path_excel: str|Path, id_col='FREE_TEXT_1', cp_col='CP_ORIG') -> pd.DataFrame:
    """Lit le fichier Trioptima et renvoie un mapping Identifier→Counterparty."""
    df = pd.read_excel(path_excel)
    df = _clean_cols(df)
    if id_col not in df.columns or cp_col not in df.columns:
        raise KeyError(f"Colonnes '{id_col}' ou '{cp_col}' manquantes dans {path_excel}")
    return (df[[id_col, cp_col]]
            .dropna(subset=[id_col])
            .rename(columns={id_col: 'Identifier', cp_col: 'Counterparty'})
            .drop_duplicates('Identifier'))

# =======================
# Étape "un jour" — base = exposures, clé = Identifier
# =======================
def day_step_apply(
    exposures: pd.DataFrame,
    merged_mapping: pd.DataFrame,
    day_col: str = "Day 1",
    include_other_inflation_swap: bool = True,
    update_duration: bool = True,   # Duration_next = Duration_t + DollarRateConvexity1pc/100 × shock_rates_dec
    update_tv: bool = True,         # TV_day = TV + TotalEffect
    return_pivot: bool = True,
    key_col: str = "Identifier",    # clé de jointure
    port_col: str = "Portfolio",    # colonne de portefeuille
):
    """
    Applique les chocs d'un jour et renvoie:
      - detailed       : lignes (Portfolio × Identifier × Market × Variable) avec Effect/Method (+ ISIN si dispo)
      - per_id         : agrégat par Portefeuille/Identifier (ISIN=first), TV_day, (Duration_next si update_duration)
      - exposures_next : copie de exposures avec TV/Duration mises à jour (selon flags)
      - pivot          : (optionnel) large des Effects ('Market :: Variable'), index = (Portefeuille, Identifier)
    """
    
    # 1) Chocs du jour (standardisés en décimal)
    shocks_day = build_daily_shocks(merged_mapping, day_col=day_col)  # contient Identifier, ISIN (POS), Market, Variable, shock_std...
    shocks_day = shocks_day.drop_duplicates(subset=[key_col, "Market", "Variable"])
    # 2) Merge en LEFT depuis exposures (la base de calcul) par Identifier et ajoute le Portefeuille
    cols_needed = [
        key_col, port_col, "AssetClass","Country","AssetID","Nominal","TVPercent","TV",
        "MacaulayDuration","Duration","DollarRateConvexity1pc","RateDelta1bp","RateVega",
        "SpreadDelta1bp","CreditVega","EquityDelta","EquityGamma","EquityVega","FXDelta","FXVega","InflationDelta1bp", "Counterparty"
    ]
    cols_needed = [c for c in cols_needed if c in exposures.columns]
    base = exposures[cols_needed].copy()
    if key_col not in base.columns or key_col not in shocks_day.columns:
        raise KeyError(f"Clé '{key_col}' absente de exposures ou du mapping de scénarios.")

    df = base.merge(shocks_day, on=key_col, how="left")  # garde les instruments sans mapping (effet=0)

    # 3) Calcul des effets par ligne (Portfolio × Identifier × Market × Variable)
    effects, methods = [], []
    for _, r in df.iterrows():
        market_raw = r.get("Market")
        if isinstance(market_raw, str):
            market = market_raw.lower()
        elif pd.notna(market_raw):
            market = str(market_raw).lower()
        else:
            market = ""
        variable = (r.get("Variable") or "")
        unit     = r.get("Unit")
        shock_std = r.get("shock_std", np.nan)

        shock_bps = _to_bps(shock_std, unit)  # pour PV01/CS01/Infl01 (en bps numériques)
        shock_dec = shock_std                 # décimal (ex: +50 bps -> +0.005)

        effect = np.nan
        method = None

        if market == "equity":
            eq_delta = r.get("EquityDelta", np.nan)
            if pd.notna(eq_delta) and pd.notna(shock_dec):
                effect = eq_delta * shock_dec
                method = "EquityDelta × shock_dec"

        elif market == "interest rates":
            if pd.notna(r.get("RateDelta1bp")) and pd.notna(shock_bps):
                effect = r["RateDelta1bp"] * shock_bps
                method = "RateDelta1bp × shock_bps"

        elif "spread" in market:  # couvre Gov Spreads / Corp Spreads (peu importe la casse)
            sp01 = r.get("SpreadDelta1bp", np.nan)
            if pd.notna(sp01) and pd.notna(shock_bps):
                effect = sp01 * shock_bps
                method = "SpreadDelta1bp × shock_bps"

        elif market == "fx":
            fx_delta = r.get("FXDelta", np.nan)
            if pd.notna(fx_delta) and pd.notna(shock_dec):
                effect = fx_delta * shock_dec
                method = "FXDelta × shock_dec"

        elif market == "other":
            # Cas spécifique demandé: Other + Variable = Inflation Swap
            if isinstance(variable, str) and "inflation swap" in variable.lower():
                infl01 = r.get("InflationDelta1bp", np.nan)
                if pd.notna(infl01) and pd.notna(shock_bps):
                    effect = infl01 * shock_bps
                    method = "InflationDelta1bp × shock_bps (Other/Inflation Swap)"

        effects.append(effect)
        methods.append(method)

    df["Effect"] = effects
    df["Method"] = methods
    df["Effect"] = df["Effect"].fillna(0.0)  # pas de mapping -> effet 0

    # 4) Agrégat par Portefeuille/Identifier (on conserve ISIN = first pour info)
    agg = {
        "TV": ("TV","first"),
        "Duration_t": ("Duration","first"),
        "DollarRateConvexity1pc": ("DollarRateConvexity1pc","first"),
        "TotalEffect": ("Effect","sum"),
        "Counterparty":("Counterparty","first")
    }
    if "ISIN" in df.columns:
        agg["ISIN"] = ("ISIN","first")

    if "AssetClass" in df.columns:
        agg["AssetClass"] = ("AssetClass","first")

    

    per_id = df.groupby([port_col, key_col], as_index=False).agg(**agg) 
    per_id["TV_day"] = per_id["TV"] + per_id["TotalEffect"]

    # 5) (option) mise à jour de la Duration : Duration_next = Duration_t + DollarRateConvexity1pc/100 × (∑choc_rates_dec)
    if update_duration:
        rates_mask = df["Market"].fillna("").str.lower().eq("interest rates")
        rates_choc_dec = (df.loc[rates_mask]
                            .groupby([port_col, key_col], as_index=False)["shock_std"]
                            .sum()
                            .rename(columns={"shock_std":"shock_rates_dec"}))
        per_id = per_id.merge(rates_choc_dec, on=[port_col, key_col], how="left")
        per_id["shock_rates_dec"] = per_id["shock_rates_dec"].fillna(0.0)
        per_id["DollarRateConvexity1pc"] = per_id["DollarRateConvexity1pc"].fillna(0.0)
        per_id["Duration_t"]      = per_id["Duration_t"].fillna(0.0)
        per_id["Duration_next"]   = per_id["Duration_t"] + (per_id['Duration_t']**2 -per_id["DollarRateConvexity1pc"] /per_id['TV']* 10000) * per_id["shock_rates_dec"]

        futures_mask = per_id["AssetClass"].isin(FUTURE_ASSET_CLASSES)
        per_id.loc[futures_mask, "Duration_next"] = per_id.loc[futures_mask, "Duration_t"]
    # 6) Construire exposures_next (mise à jour TV/Duration par (Portefeuille, Identifier))


    exposures_next = exposures.copy()
    merge_cols = [port_col, key_col]
    update_cols = merge_cols + ["TV_day"]
    if update_duration:
        update_cols.append("Duration_next")
        
    exposures_next = exposures_next.merge(
        per_id[update_cols],
        on=merge_cols,
        how="left",
    )
    exposures_next["TV"] = exposures_next["TV_day"].fillna(exposures_next["TV"])
    if update_duration and "Duration_next" in exposures_next.columns:
        futures_mask = exposures_next["AssetClass"].isin(FUTURE_ASSET_CLASSES)
        duration_updates = exposures_next["Duration_next"].fillna(exposures_next["Duration"])
        exposures_next.loc[~futures_mask, "Duration"] = duration_updates.loc[~futures_mask]

    exposures_next = exposures_next.drop(columns=[c for c in ["TV_day", "Duration_next"] if c in exposures_next.columns])


    return df, per_id, exposures_next


In [2]:

# 1) Charger mapping & scénarios
pos_raw, scen_raw = load_mapping(EXCEL_MAPPING_PATH)
scen_raw=scen_raw.iloc[:29,:]


# 2) Charger exposures (greeks) + QC
exposures, qc = load_exposures(EXPOSURES_PATH, return_qc=True)
print_qc_report(qc)

# Ajouter les contreparties depuis Trioptima
cp_map = load_counterparty_mapping(TRIOPTIMA_PATH)
exposures = exposures.merge(cp_map, on='Identifier', how='left')

# Limiter POS au périmètre d'exposures avant melt
pos_subset = exposures[['Identifier','Counterparty']].merge(pos_raw, on='Identifier', how='left')
# 3) Préparer mapping restreint & scénarios 
pos_long = melt_pos(pos_subset)
scen     = prepare_scenarios(scen_raw)
merged   = merge_pos_scen(pos_long, scen)          # mapping POS ↔ scénarios


# 4) DAY 1
d1_det, d1_id, exp_d2 = day_step_apply(
    exposures=exposures,
    merged_mapping=merged,
    day_col="Day 1",
    include_other_inflation_swap=INCLUDE_OTHER_INFLATION_SWAP,
    update_duration=True,
    update_tv=True,
    return_pivot=True,
    key_col="Identifier",
    port_col="Portfolio",
)



# Agrégation des TV et remise à zéro pour certains futures
updated_exp, futures_tv, balances, decisions, alerts = process_pv_after_day_1(exp_d2)
print("TV avant remise à zéro par AssetClass (futures):")
with pd.option_context("display.float_format", "{:.2f}".format):
        print(futures_tv)









📄 Fichier : C:\Users\abenjelloun\OneDrive - Cooperactions\GAM-E-Risk Perf - RMP\1.PROD\1.REGLEMENTAIRE\14.Stress Test AMF (JB)\Production\Périmètre et positions\GROUPAMA-BreakoutsOverTime-2025-03-31.csv
📦 Lignes brutes : 222
🧹 Lignes 'TOTAL' supprimées : 2
✅ Lignes après filtre : 220
🔧 NaN → 0 (sensis) :
  - RateDelta1bp          : 1
  - SpreadDelta1bp        : 40
  - InflationDelta1bp     : 191
  - FXDelta               : 181

⏱  Duration NaN → 0 : 10
📐 DollarRateConvexity1pc NaN → 0 : 24
TV avant remise à zéro par AssetClass (futures):
            AssetClass  TV_before_reset
0          Bond Future       3686943.19
1  Equity Index Future             0.00


In [7]:
decisions.to_csv("decisions.csv",sep=";",decimal=".",index=False)
d1_id.to_csv("d1_id.csv",sep=";",decimal=".",index=False)
d1_det.to_csv("d1_det.csv",sep=";",decimal=".",index=False)
exp_d2.to_csv("exp_d2.csv",sep=";",decimal=".",index=False)

In [4]:
COLUMNS_TO_KEEP = [
    'AssetType',
    'Sector1',
    'Seniority',
    'CompositeBroadRating',
    'MaturityDate',
    'Maturity',
    'Maturity Band',
    'EffectiveMaturityDate',
    'LiquidityScore',
    'Country',
    '{Class_Rating}',
]
def merge_day1_positions(exposures: pd.DataFrame, day1_per_id: pd.DataFrame) -> pd.DataFrame:
    """Combine current exposures with Day‑1 results.

    Parameters
    ----------
    exposures : pd.DataFrame
        DataFrame containing the current positions. Must include an
        ``Identifier`` column so that classification fields can be merged.
    day1_per_id : pd.DataFrame
        ``per_id`` DataFrame returned by :func:`day_step_apply`. If the
        identifier column is named ``Identifier`` it will be normalised to
        ``d1_id``.

    Returns
    -------
    pd.DataFrame
        Day‑1 positions enriched with classification columns and a
        ``TV_change`` column equal to ``TV - TV_day`` when both are available.
    """

    if 'Identifier' in day1_per_id.columns and 'd1_id' not in day1_per_id.columns:
        day1_per_id = day1_per_id.rename(columns={'Identifier': 'd1_id'})

    class_cols = [c for c in COLUMNS_TO_KEEP if c in exposures.columns]
    class_df = exposures[['Identifier'] + class_cols].drop_duplicates('Identifier')
    class_df = class_df.rename(columns={'Identifier': 'd1_id'})

    merged = day1_per_id.merge(class_df, on='d1_id', how='left')

    if 'TV' in merged.columns and 'TV_day' in merged.columns:
        merged['TV_change'] = merged['TV'] - merged['TV_day']

    return merged

def aggregate_positions(df: pd.DataFrame) -> pd.DataFrame:
    """Aggregate positions by the available classification columns.

    All numeric trade value columns found in the dataframe are summed. The
    function looks for ``TV`` (current value), ``TV_day`` (day-one value) and the
    computed ``TV_change``. When none of these columns are present a simple
    count per group is returned instead.
    """

    group_cols = [col for col in COLUMNS_TO_KEEP if col in df.columns]

    agg_spec = {}
    for col in ('TV', 'TV_day', 'TV_change'):
        if col in df.columns:
            agg_spec[col] = 'sum'

    if agg_spec:
        agg = df.groupby(group_cols, dropna=False).agg(agg_spec).reset_index()
    else:
        agg = df.groupby(group_cols, dropna=False).size().reset_index(name='count')

    return agg

def process_positions_df(exposures: pd.DataFrame, d1_id: pd.DataFrame, aggregate: bool = False) -> pd.DataFrame:
    #Merge Day 1 results from ``day_step_apply`` with exposures.
    df = merge_day1_positions(exposures, d1_id)
    if aggregate:
        df = aggregate_positions(df)
    return df


In [5]:
# Country groupings for government bond aggregation
EUROZONE_LOW_RISK = {
    'Austria', 'Belgium', 'Finland', 'Germany', 'Ireland', 'Latvia', 'Luxembourg', 'Netherlands', 'Slovenia'
}
EUROZONE_MEDIUM_RISK = {
    'Croatia', 'Cyprus', 'France', 'Lithuania', 'Malta', 'Portugal', 'Slovakia'
}
EUROZONE_HIGH_RISK = {
    'Greece', 'Italy', 'Spain'
}

Emerging_MARKETS = {
    'Argentina', 'Brazil', 'Chile', 'China', 'Colombia', 'India', 'Indonesia', 'Mexico', 'Peru', 'South Africa', 'Turkey'
}

Advanced_economics_MARKETS = {'United States', 'Canada' ,
                               'United Kingdom', 'Germany', 'France', 'Italy', 'Spain', 'Netherlands', 'Sweden', 'Switzerland', 'Norway', 'Austria', 'Belgium', 'Denmark', 'Finland', 'Ireland', 'Portugal', 'Greece', 'Czech Republic', 'Slovakia', 'Slovenia', 'Estonia', 'Latvia', 'Lithuania', 'Luxembourg', 'Croatia', 'Cyprus', 'Malta','Andorra', 'San Marino',
                              'Japan', 'South Korea', 'Australia', 'New Zealand', 'Singapore', 'Hong Kong SAR', 'Taiwan', 'Macao SAR',
                            'Israel', 'Iceland', 'Puerto Rico' }